In [29]:
%pip install xgboost

import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
import xgboost as xgb
from scipy.stats import entropy
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import time
from tqdm import tqdm
from google.colab import drive

drive.mount('/content/drive/')
%cd /content/drive/MyDrive/Colab Notebooks/Katabatic/MedGAN/poker/

# Suppress warnings
warnings.filterwarnings("ignore")

# Setting random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Check if CUDA is available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/Katabatic/MedGAN/poker
Using device: cpu


In [30]:
# Load and preprocess data
def load_poker_data(train_path, test_path):
    # Column names based on the dataset description
    column_names = ['S1', 'C1', 'S2', 'C2', 'S3', 'C3', 'S4', 'C4', 'S5', 'C5', 'CLASS']

    # Load data
    df_train = pd.read_csv(train_path, header=None, names=column_names)
    df_test = pd.read_csv(test_path, header=None, names=column_names)

    print(f"Training data shape: {df_train.shape}")
    print(f"Testing data shape: {df_test.shape}")

    # Split features and target
    X_train = df_train.drop('CLASS', axis=1)
    y_train = df_train['CLASS']
    X_test = df_test.drop('CLASS', axis=1)
    y_test = df_test['CLASS']

    # Create preprocessing pipeline - for poker hands, we'll use one-hot encoding for suits
    # and normalize the card ranks

    # Define suit columns and rank columns
    suit_cols = ['S1', 'S2', 'S3', 'S4', 'S5']
    rank_cols = ['C1', 'C2', 'C3', 'C4', 'C5']

    # Create transformers
    suit_transformer = Pipeline(steps=[
        ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
    ])

    rank_transformer = Pipeline(steps=[
        ('scaler', StandardScaler())
    ])

    preprocessor = ColumnTransformer(
        transformers=[
            ('suit', suit_transformer, suit_cols),
            ('rank', rank_transformer, rank_cols)
        ])

    # Fit and transform the data
    X_train_transformed = preprocessor.fit_transform(X_train)
    X_test_transformed = preprocessor.transform(X_test)

    # Get feature names
    suit_feature_names = preprocessor.named_transformers_['suit'].named_steps['onehot'].get_feature_names_out(suit_cols)
    all_feature_names = list(suit_feature_names) + list(rank_cols)

    print(f"Processed training data shape: {X_train_transformed.shape}")
    print(f"Processed testing data shape: {X_test_transformed.shape}")

    return X_train_transformed, y_train, X_test_transformed, y_test, preprocessor, all_feature_names

# Custom dataset class
class PokerDataset(Dataset):
    def __init__(self, features, labels=None):
        self.features = torch.tensor(features, dtype=torch.float32)
        if labels is not None:
            self.labels = torch.tensor(labels, dtype=torch.float32).view(-1, 1)
        else:
            self.labels = None

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        if self.labels is not None:
            return self.features[idx], self.labels[idx]
        else:
            return self.features[idx]

# Autoencoder for MedGAN
class Autoencoder(nn.Module):
    def __init__(self, input_dim, encoding_dim=128):
        super(Autoencoder, self).__init__()

        # Encoder
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Linear(256, encoding_dim),
            nn.BatchNorm1d(encoding_dim),
            nn.ReLU()
        )

        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(encoding_dim, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Linear(512, input_dim),
            nn.Tanh()  # Output range (-1, 1)
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

    def encode(self, x):
        return self.encoder(x)

# Generator for MedGAN
class Generator(nn.Module):
    def __init__(self, latent_dim, hidden_dim=128, output_dim=128):
        super(Generator, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),

            nn.Linear(hidden_dim, hidden_dim*2),
            nn.BatchNorm1d(hidden_dim*2),
            nn.ReLU(),

            nn.Linear(hidden_dim*2, output_dim),
            nn.Tanh()  # Output in range (-1, 1)
        )

    def forward(self, z):
        return self.model(z)

# Discriminator for MedGAN
class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),

            nn.Linear(256, 128),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),

            nn.Linear(128, 64),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),

            nn.Linear(64, 1),
            nn.Sigmoid()  # Output probability
        )

    def forward(self, x):
        return self.model(x)

# MedGAN Implementation
class MedGAN:
    def __init__(self, data_dim, encoding_dim=128, latent_dim=128,
                 batch_size=128, autoencoder_epochs=100):
        self.data_dim = data_dim
        self.encoding_dim = encoding_dim
        self.latent_dim = latent_dim
        self.batch_size = batch_size

        # Initialize networks
        self.autoencoder = Autoencoder(data_dim, encoding_dim).to(device)
        self.generator = Generator(latent_dim, hidden_dim=256, output_dim=encoding_dim).to(device)
        self.discriminator = Discriminator(data_dim).to(device)

        # Setup optimizers
        self.ae_optimizer = optim.Adam(self.autoencoder.parameters(), lr=0.001)
        self.g_optimizer = optim.Adam(self.generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
        self.d_optimizer = optim.Adam(self.discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))

        # Loss functions
        self.ae_criterion = nn.MSELoss()
        self.gan_criterion = nn.BCELoss()

        # For tracking progress
        self.ae_losses = []
        self.g_losses = []
        self.d_losses = []
        self.autoencoder_epochs = autoencoder_epochs

    def pretrain_autoencoder(self, data_loader, epochs=None):
        """Pretrain the autoencoder"""
        if epochs is None:
            epochs = self.autoencoder_epochs

        print(f"Pretraining autoencoder for {epochs} epochs...")

        self.autoencoder.train()
        for epoch in range(epochs):
            epoch_loss = 0
            num_batches = 0

            for real_data, _ in data_loader:
                real_data = real_data.to(device)

                # Forward pass
                reconstructed = self.autoencoder(real_data)
                loss = self.ae_criterion(reconstructed, real_data)

                # Backward pass
                self.ae_optimizer.zero_grad()
                loss.backward()
                self.ae_optimizer.step()

                epoch_loss += loss.item()
                num_batches += 1

            avg_loss = epoch_loss / num_batches
            self.ae_losses.append(avg_loss)

            if (epoch + 1) % 10 == 0 or epoch == 0:
                print(f"Autoencoder Epoch {epoch+1}/{epochs} | Loss: {avg_loss:.6f}")

        print("Autoencoder pretraining complete!")

    def train_gan(self, data_loader, epochs=100, save_interval=10):
        """Train the GAN after pretraining the autoencoder"""
        print(f"Training MedGAN for {epochs} epochs...")

        # Prepare labels for real and fake data
        real_label = 1.0
        fake_label = 0.0

        for epoch in range(epochs):
            d_loss_total = 0
            g_loss_total = 0
            num_batches = 0

            for real_data, _ in data_loader:
                batch_size = real_data.size(0)
                real_data = real_data.to(device)

                # ---------------------
                # Train Discriminator
                # ---------------------
                self.d_optimizer.zero_grad()

                # Real data
                real_output = self.discriminator(real_data)
                d_real_loss = self.gan_criterion(
                    real_output,
                    torch.full((batch_size, 1), real_label, device=device)
                )

                # Fake data
                noise = torch.randn(batch_size, self.latent_dim, device=device)
                fake_encoded = self.generator(noise)
                fake_data = self.autoencoder.decoder(fake_encoded)
                fake_output = self.discriminator(fake_data.detach())
                d_fake_loss = self.gan_criterion(
                    fake_output,
                    torch.full((batch_size, 1), fake_label, device=device)
                )

                # Combined loss
                d_loss = d_real_loss + d_fake_loss
                d_loss.backward()
                self.d_optimizer.step()

                # ---------------------
                # Train Generator
                # ---------------------
                self.g_optimizer.zero_grad()

                # Generate new fake data
                noise = torch.randn(batch_size, self.latent_dim, device=device)
                fake_encoded = self.generator(noise)
                fake_data = self.autoencoder.decoder(fake_encoded)
                fake_output = self.discriminator(fake_data)

                # Generator loss
                g_loss = self.gan_criterion(
                    fake_output,
                    torch.full((batch_size, 1), real_label, device=device)
                )
                g_loss.backward()
                self.g_optimizer.step()

                d_loss_total += d_loss.item()
                g_loss_total += g_loss.item()
                num_batches += 1

            # Calculate average losses
            avg_d_loss = d_loss_total / num_batches
            avg_g_loss = g_loss_total / num_batches

            self.d_losses.append(avg_d_loss)
            self.g_losses.append(avg_g_loss)

            if (epoch + 1) % save_interval == 0 or epoch == 0:
                print(f"GAN Epoch {epoch+1}/{epochs} | D Loss: {avg_d_loss:.6f} | G Loss: {avg_g_loss:.6f}")

        print("MedGAN training complete!")

    def generate_samples(self, num_samples):
        """Generate synthetic samples"""
        self.generator.eval()
        self.autoencoder.eval()

        batches = []
        remaining = num_samples

        while remaining > 0:
            batch_size = min(remaining, self.batch_size)
            noise = torch.randn(batch_size, self.latent_dim, device=device)

            with torch.no_grad():
                fake_encoded = self.generator(noise)
                fake_data = self.autoencoder.decoder(fake_encoded)

            batches.append(fake_data.cpu().numpy())
            remaining -= batch_size

        synthetic_data = np.vstack(batches)

        return synthetic_data

    def save_model(self, path):
        """Save trained models"""
        torch.save({
            'autoencoder_state_dict': self.autoencoder.state_dict(),
            'generator_state_dict': self.generator.state_dict(),
            'discriminator_state_dict': self.discriminator.state_dict(),
            'ae_optimizer_state_dict': self.ae_optimizer.state_dict(),
            'g_optimizer_state_dict': self.g_optimizer.state_dict(),
            'd_optimizer_state_dict': self.d_optimizer.state_dict(),
        }, path)

    def load_model(self, path):
        """Load trained models"""
        checkpoint = torch.load(path)
        self.autoencoder.load_state_dict(checkpoint['autoencoder_state_dict'])
        self.generator.load_state_dict(checkpoint['generator_state_dict'])
        self.discriminator.load_state_dict(checkpoint['discriminator_state_dict'])
        self.ae_optimizer.load_state_dict(checkpoint['ae_optimizer_state_dict'])
        self.g_optimizer.load_state_dict(checkpoint['g_optimizer_state_dict'])
        self.d_optimizer.load_state_dict(checkpoint['d_optimizer_state_dict'])

# Post-process generated data to make it valid poker hands
def post_process_poker_data(synthetic_data, preprocessor):
    """
    Post-process generated data to ensure it can be translated back into valid poker hands
    """
    # Get the number of suit features (one-hot encoded)
    n_suit_features = len(preprocessor.named_transformers_['suit'].named_steps['onehot'].get_feature_names_out(['S1', 'S2', 'S3', 'S4', 'S5']))

    # Split the synthetic data into suits and ranks
    synthetic_suits = synthetic_data[:, :n_suit_features]
    synthetic_ranks = synthetic_data[:, n_suit_features:]

    # For each card's suit (one-hot encoded), take the max value to get the most likely suit
    num_cards = 5
    n_suits = 4  # Hearts, Spades, Diamonds, Clubs

    processed_suits = np.zeros((len(synthetic_data), num_cards))

    for i in range(num_cards):
        one_hot_indices = np.argmax(synthetic_suits[:, i*n_suits:(i+1)*n_suits], axis=1)
        processed_suits[:, i] = one_hot_indices + 1  # Add 1 to match original encoding (1-4)

    # Inverse transform the rank data
    processed_ranks = preprocessor.named_transformers_['rank'].named_steps['scaler'].inverse_transform(synthetic_ranks)

    # Clip and round ranks to valid values (1-13)
    processed_ranks = np.clip(np.round(processed_ranks), 1, 13)

    # Combine suits and ranks
    processed_data = np.zeros((len(synthetic_data), num_cards * 2))

    for i in range(num_cards):
        processed_data[:, 2*i] = processed_suits[:, i]       # Suit
        processed_data[:, 2*i+1] = processed_ranks[:, i]     # Rank

    return processed_data



In [31]:
# Evaluation Metrics

# 1. Machine Learning Utility (TSTR)
def evaluate_tstr(real_data, synthetic_data, real_labels, random_state=42):
    """
    Train classifiers on synthetic data and test on real data (TSTR)
    Returns accuracy for each classifier
    """
    # Train-test split for real data
    X_train, X_test, y_train, y_test = train_test_split(
        real_data, real_labels, test_size=0.2, random_state=random_state
    )

    # Synthetic data (all used for training)
    X_synth = synthetic_data

    # Ensure proper dimensions for labels
    if isinstance(y_train, pd.Series):
        y_train = y_train.values

    # Create synthetic labels based on real distribution
    # For poker hands, we need to represent the distribution of classes (0-9)
    class_distribution = np.bincount(y_test.astype(int), minlength=10).astype(float)
    class_distribution += 1e-6  # Add small value to ensure all classes have a non-zero probability
    class_distribution /= class_distribution.sum()  # Normalize

    # Sample synthetic labels using this adjusted distribution
    np.random.seed(random_state)
    y_synth = np.random.choice(range(10), size=len(X_synth), p=class_distribution)

    # Define classifiers
    classifiers = {
        'Logistic Regression': LogisticRegression(max_iter=1000, random_state=random_state),
        'MLP': MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=500, random_state=random_state),
        'Random Forest': RandomForestClassifier(n_estimators=100, random_state=random_state),
        'XGBoost': xgb.XGBClassifier(n_estimators=100, random_state=random_state)
    }

    results = {}

    for name, clf in classifiers.items():
        # Train on synthetic data
        clf.fit(X_synth, y_synth)

        # Test on real data
        y_pred = clf.predict(X_test)

        # Calculate metrics
        accuracy = accuracy_score(y_test, y_pred)

        # We use weighted f1 since poker hands dataset is heavily imbalanced
        f1 = f1_score(y_test, y_pred, average='weighted')

        results[name] = {
            'accuracy': accuracy,
            'f1_score': f1
        }

    return results

# 2. Statistical Similarity
def jensen_shannon_divergence(p, q):
    """
    Calculate Jensen-Shannon Divergence between distributions p and q
    """
    # Ensure p and q are normalized
    p = p / np.sum(p)
    q = q / np.sum(q)

    m = 0.5 * (p + q)

    # Calculate JSD
    jsd = 0.5 * (entropy(p, m) + entropy(q, m))

    return jsd

def wasserstein_distance(p, q):
    """
    Calculate 1D Wasserstein distance (Earth Mover's Distance)
    """
    from scipy.stats import wasserstein_distance

    return wasserstein_distance(p, q)

def evaluate_statistical_similarity(real_data, synthetic_data, feature_names):
    """
    Calculate statistical similarity metrics between real and synthetic data
    """
    results = {'JSD': {}, 'WD': {}}

    # Calculate metrics for each feature
    for i in range(real_data.shape[1]):
        feature_name = feature_names[i] if i < len(feature_names) else f"feature_{i}"

        # Get feature values
        real_values = real_data[:, i]
        synth_values = synthetic_data[:, i]

        # Calculate histogram (discrete distribution)
        hist_bins = min(50, len(np.unique(real_values)))

        hist_real, bin_edges = np.histogram(real_values, bins=hist_bins, density=True)
        hist_synth, _ = np.histogram(synth_values, bins=bin_edges, density=True)

        # Add a small epsilon to avoid division by zero
        epsilon = 1e-10
        hist_real = hist_real + epsilon
        hist_synth = hist_synth + epsilon

        # Calculate JSD
        jsd = jensen_shannon_divergence(hist_real, hist_synth)
        results['JSD'][feature_name] = jsd

        # Calculate Wasserstein Distance
        wd = wasserstein_distance(real_values, synth_values)
        results['WD'][feature_name] = wd

    # Calculate average metrics
    results['JSD_avg'] = np.mean(list(results['JSD'].values()))
    results['WD_avg'] = np.mean(list(results['WD'].values()))

    return results

def plot_loss_curves(model):
    """
    Plot the loss curves for the autoencoder, generator, and discriminator
    """
    plt.figure(figsize=(12, 8))

    # Plot autoencoder loss
    plt.subplot(2, 1, 1)
    plt.plot(model.ae_losses, label='Autoencoder Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('MedGAN Autoencoder Training Loss')
    plt.legend()
    plt.grid(True)

    # Plot GAN losses
    plt.subplot(2, 1, 2)
    plt.plot(model.g_losses, label='Generator Loss')
    plt.plot(model.d_losses, label='Discriminator Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('MedGAN Adversarial Training Loss')
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.savefig('poker_medgan_loss_curves.png')
    plt.close()

def plot_feature_distributions(real_data, synthetic_data, feature_names, n_features=10):
    """
    Plot distributions of real vs synthetic data for selected features
    """
    if n_features > len(feature_names):
        n_features = len(feature_names)

    # Select a subset of features to visualize
    selected_indices = np.random.choice(range(len(feature_names)), size=n_features, replace=False)

    plt.figure(figsize=(15, 20))
    for i, idx in enumerate(selected_indices):
        feature_name = feature_names[idx]

        plt.subplot(n_features, 1, i+1)

        # Get feature values
        real_values = real_data[:, idx]
        synth_values = synthetic_data[:, idx]

        # Plot histograms
        sns.histplot(real_values, kde=True, stat="density", label="Real", alpha=0.6, color="blue")
        sns.histplot(synth_values, kde=True, stat="density", label="Synthetic", alpha=0.6, color="red")

        plt.title(f"Distribution for {feature_name}")
        plt.legend()

    plt.tight_layout()
    plt.savefig('poker_feature_distributions.png')
    plt.close()

def plot_class_distribution(real_labels, synthetic_labels):
    """
    Plot the class distribution of real vs synthetic data
    """
    plt.figure(figsize=(12, 6))

    # Get class counts
    real_class_counts = np.bincount(real_labels.astype(int), minlength=10)
    synth_class_counts = np.bincount(synthetic_labels.astype(int), minlength=10)

    # Normalize
    real_class_dist = real_class_counts / np.sum(real_class_counts)
    synth_class_dist = synth_class_counts / np.sum(synth_class_counts)

    # Define class names
    class_names = [
        'Nothing in hand',
        'One pair',
        'Two pairs',
        'Three of a kind',
        'Straight',
        'Flush',
        'Full house',
        'Four of a kind',
        'Straight flush',
        'Royal flush'
    ]

    # Plot
    bar_width = 0.35
    x = np.arange(10)

    plt.bar(x - bar_width/2, real_class_dist, bar_width, label='Real Data', color='blue', alpha=0.7)
    plt.bar(x + bar_width/2, synth_class_dist, bar_width, label='Synthetic Data', color='red', alpha=0.7)

    plt.xlabel('Poker Hand')
    plt.ylabel('Proportion')
    plt.title('Class Distribution: Real vs Synthetic Poker Hands')
    plt.xticks(x, class_names, rotation=45, ha='right')
    plt.legend()
    plt.tight_layout()
    plt.savefig('poker_class_distribution.png')
    plt.close()

# Main function
def main():
    # File paths
    train_path = 'data/poker-hand-training-true.csv'
    test_path = 'data/poker-hand-testing.csv'

    # Load and preprocess data
    X_train, y_train, X_test, y_test, preprocessor, feature_names = load_poker_data(train_path, test_path)

    # Create dataset and dataloader
    train_dataset = PokerDataset(X_train, y_train)
    train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)

    # Initialize MedGAN
    data_dim = X_train.shape[1]
    encoding_dim = 128
    latent_dim = 100

    print(f"Data dimension: {data_dim}")
    print(f"Encoding dimension: {encoding_dim}")
    print(f"Latent dimension: {latent_dim}")

    medgan = MedGAN(data_dim, encoding_dim, latent_dim, batch_size=128, autoencoder_epochs=50)

    # Step 1: Pretrain the autoencoder
    medgan.pretrain_autoencoder(train_loader)

    # Step 2: Train the GAN
    medgan.train_gan(train_loader, epochs=100, save_interval=10)

    # Save the model
    medgan.save_model('poker_medgan_model.pt')

    # Plot loss curves
    plot_loss_curves(medgan)

    # Generate synthetic data
    num_samples = 1000
    print(f"Generating {num_samples} synthetic samples...")
    synthetic_data_raw = medgan.generate_samples(num_samples)

    # Post-process the synthetic data to make it valid poker hands
    synthetic_data_processed = post_process_poker_data(synthetic_data_raw, preprocessor)

    # Generate synthetic labels using a classifier trained on real data
    clf = RandomForestClassifier(n_estimators=100, random_state=42)
    clf.fit(X_train, y_train)
    synthetic_labels = clf.predict(synthetic_data_raw)

    # Plot class distribution
    plot_class_distribution(y_train, synthetic_labels)

    # Statistical similarity evaluation
    print("Evaluating statistical similarity...")
    stat_results = evaluate_statistical_similarity(X_train, synthetic_data_raw, feature_names)

    print("\nJensen-Shannon Divergence (average):", stat_results['JSD_avg'])
    print("Wasserstein Distance (average):", stat_results['WD_avg'])

    # Machine Learning Utility (TSTR) evaluation
    print("\nEvaluating Machine Learning Utility (TSTR)...")
    tstr_results = evaluate_tstr(X_train, synthetic_data_raw, y_train)

    print("\nTSTR Results:")
    for clf_name, metrics in tstr_results.items():
        print(f"{clf_name}: Accuracy = {metrics['accuracy']:.4f}, F1 Score = {metrics['f1_score']:.4f}")

    # Plot feature distributions
    plot_feature_distributions(X_train, synthetic_data_raw, feature_names)

    print("\nEvaluation complete! Check the output directory for plots and saved model.")


In [32]:
if __name__ == "__main__":
    main()

Training data shape: (25010, 11)
Testing data shape: (1000000, 11)
Processed training data shape: (25010, 25)
Processed testing data shape: (1000000, 25)
Data dimension: 25
Encoding dimension: 128
Latent dimension: 100
Pretraining autoencoder for 50 epochs...
Autoencoder Epoch 1/50 | Loss: 0.056696
Autoencoder Epoch 10/50 | Loss: 0.017806
Autoencoder Epoch 20/50 | Loss: 0.016816
Autoencoder Epoch 30/50 | Loss: 0.016778
Autoencoder Epoch 40/50 | Loss: 0.016382
Autoencoder Epoch 50/50 | Loss: 0.016206
Autoencoder pretraining complete!
Training MedGAN for 100 epochs...
GAN Epoch 1/100 | D Loss: 1.273534 | G Loss: 0.781497
GAN Epoch 10/100 | D Loss: 1.163367 | G Loss: 0.949234
GAN Epoch 20/100 | D Loss: 0.949228 | G Loss: 1.291191
GAN Epoch 30/100 | D Loss: 0.889354 | G Loss: 1.410466
GAN Epoch 40/100 | D Loss: 0.852112 | G Loss: 1.472878
GAN Epoch 50/100 | D Loss: 0.804739 | G Loss: 1.570688
GAN Epoch 60/100 | D Loss: 0.714435 | G Loss: 1.774080
GAN Epoch 70/100 | D Loss: 0.594836 | G Los